In [1]:
from keras import regularizers
from tensorflow import keras
import datetime

import db
import numpy as np
import pickle
import matplotlib.pyplot as plt
import random

from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN, GRU, Embedding
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

dbConn = db.DataBase()

In [2]:
def get_win_good_with(gods, matches):
    # synergy win rate
    gods = list(map(lambda x: x[0], gods))
    good_with = np.zeros((len(gods), len(gods))) * 0.5
    good_with_num_matches = np.zeros((len(gods), len(gods)))
    for match in matches:
        for god_1_id in match[:5]:
            i = gods.index(god_1_id)
            for god_2_id in match[:5]:
                if god_1_id != god_2_id:
                    j = gods.index(god_2_id)
                    good_with[i, j] += 1 if match[len(match) - 1] == 1 else 0
                    good_with_num_matches[i, j] += 1

        for god_1_id in match[5:10]:
            i = gods.index(god_1_id)
            for god_2_id in match[5:10]:
                if god_1_id != god_2_id:
                    j = gods.index(god_2_id)
                    good_with[i, j] += 1 if match[len(match) - 1] == 2 else 0
                    good_with_num_matches[i, j] += 1

    win_good_with = np.divide(good_with, good_with_num_matches, out=np.zeros_like(good_with), where=good_with_num_matches != 0)
    pickle.dump(win_good_with, open("./pickles/win_good_with.p", "wb"))

    return win_good_with


def get_win_good_against(gods, matches):
    # countering win rate
    gods = list(map(lambda x: x[0], gods))
    good_against = np.zeros((len(gods), len(gods))) * 0.5
    good_against_num_matches = np.zeros((len(gods), len(gods)))
    for match in matches:
        for god_1_id in match[:5]:
            i = gods.index(god_1_id)
            for god_2_id in match[5:10]:
                j = gods.index(god_2_id)

                if match[len(match) - 1] == 1:
                    good_against[i, j] += 1
                else:
                    good_against[j, i] += 1

                good_against_num_matches[i, j] += 1
                good_against_num_matches[j, i] += 1

    win_good_against = np.divide(good_against, good_against_num_matches, out=np.zeros_like(good_against), where=good_against_num_matches != 0)
    pickle.dump(win_good_against, open("./pickles/win_good_against.p", "wb"))

    return win_good_against


def get_data(gods):
    matches = dbConn.select("SELECT "
                            "t1_god1, t1_god2, t1_god3, t1_god4, t1_god5, "
                            "t2_god1, t2_god2, t2_god3, t2_god4, t2_god5, "
                            "ban1, ban2, ban3, ban4, ban5, ban6, ban7, ban8, ban9, ban10, "
                            "win FROM Match", ())

    win_good_with = get_win_good_with(gods, matches)
    win_good_against = get_win_good_against(gods, matches)

    gods = list(map(lambda x: x[0], gods))
    X = []
    y = []
    sum_god = [0] * len(gods)
    for match in matches:
        #features = list(map(lambda god: 1 if god in match[:5] else -1 if god in match[5:10] else 0, gods))
        features_team_1 = list(map(lambda god: 1 if god in match[:5] else 0, gods))
        features_team_2 = list(map(lambda god: 1 if god in match[5:10] else 0, gods))
        features = features_team_1 + features_team_2

        s_team_1 = 0
        s_team_2 = 0
        n_synergy_team_1 = 0
        n_synergy_team_2 = 0
        counter = 0
        n_counter = 0
        for god_1_id in match[:5]:
            i = gods.index(god_1_id)
            for god_2_id in match[:5]:
                if god_1_id != god_2_id:
                    j = gods.index(god_2_id)
                    s_team_1 += win_good_with[i, j]
                    n_synergy_team_1 += 1

        for god_1_id in match[5:10]:
            i = gods.index(god_1_id)
            for god_2_id in match[5:10]:
                if god_1_id != god_2_id:
                    j = gods.index(god_2_id)
                    s_team_2 += win_good_with[i, j]
                    n_synergy_team_2 += 1

            for god_2_id in match[:5]:
                if god_1_id != god_2_id:
                    j = gods.index(god_2_id)
                    counter += win_good_against[i, j]
                    n_counter += 1

        s_team_1 = s_team_1 / n_synergy_team_1
        s_team_2 = s_team_2 / n_synergy_team_2
        features.append(s_team_1 / (s_team_1 + s_team_2))
        features.append(counter / n_counter)

        X.append(features)
        y.append(match[len(match) - 1] - 1)

    # sum_god = [sum(x) for x in zip(sum_god, features_team_1[:len(features_team_1)-2])]
    # sum_god = [sum(x) for x in zip(sum_god, features_team_2[:len(features_team_2)-2])]
    # sum_god.sort()
    # print(sum_god)

    # randomly shuffle cases to avoid overfitting
    #c = list(zip(X, y))
    #random.shuffle(c)
    #X, y = zip(*c)

    return X, y

In [3]:
def build_model():
    gods = dbConn.select("SELECT * FROM God ORDER BY name", ())
    X, y = get_data(gods)

    print(len(X), "samples")
    print(len(X[0]), "features")

    # Split the dataset into training and test dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    # define the keras model
    #model = Sequential()
    #model.add(Dense(8, input_dim=len(X[0]), activation='elu', kernel_regularizer=regularizers.L2(0.001)))
    #model.add(Dense(8, activation='elu', kernel_regularizer=regularizers.L2(0.001)))
    #model.add(Dense(8, activation='elu', kernel_regularizer=regularizers.L2(0.001)))
    #model.add(Dense(8, activation='elu', kernel_regularizer=regularizers.L2(0.001)))
    #model.add(Dense(8, activation='elu', kernel_regularizer=regularizers.L2(0.001)))
    #model.add(Dense(1))
    
    model = Sequential()
    model.add(Dense(len(X[0]), input_dim=len(X[0]), activation='elu', kernel_regularizer=regularizers.L2(0.001)))
    model.add(Dense(300, activation='elu', kernel_regularizer=regularizers.L2(0.001)))
    model.add(Dense(1, activation='sigmoid'))
    
    #model = Sequential()
    #model.add(Embedding(input_dim=len(X[0]), output_dim=32))
    #model.add(GRU(64, return_sequences=True))
    #model.add(SimpleRNN(32))
    #model.add(Dense(1))


    # compile the keras model
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(
        learning_rate=0.0001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
    ), metrics=['accuracy'])

    # fit the keras model on the dataset
    model.fit(X_train, y_train, epochs=200, batch_size=10, validation_data=(X_test, y_test), shuffle=True, callbacks=[tensorboard_callback])
    model.save("./models/dnn.h5")

    # evaluate the keras model
    _, accuracy = model.evaluate(X_test, y_test)
    print('Accuracy: %.2f' % (accuracy * 100))

log_dir = "logs/fit/dnn" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

build_model()


59170 samples
244 features
Epoch 1/200
4142/4142 [==============================] - 30s 7ms/step - loss: 0.9868 - accuracy: 0.5546 - val_loss: 0.8556 - val_accuracy: 0.5611
Epoch 2/200
4142/4142 [==============================] - 13s 3ms/step - loss: 0.7906 - accuracy: 0.5715 - val_loss: 0.7463 - val_accuracy: 0.5649
Epoch 3/200
4142/4142 [==============================] - 15s 4ms/step - loss: 0.7219 - accuracy: 0.5733 - val_loss: 0.7079 - val_accuracy: 0.5653
Epoch 4/200
4142/4142 [==============================] - 14s 3ms/step - loss: 0.6981 - accuracy: 0.5761 - val_loss: 0.6959 - val_accuracy: 0.5666
Epoch 5/200
4142/4142 [==============================] - 12s 3ms/step - loss: 0.6899 - accuracy: 0.5766 - val_loss: 0.6903 - val_accuracy: 0.5682
Epoch 6/200
4142/4142 [==============================] - 13s 3ms/step - loss: 0.6860 - accuracy: 0.5767 - val_loss: 0.6887 - val_accuracy: 0.5667
Epoch 7/200
4142/4142 [==============================] - 14s 3ms/step - loss: 0.6838 - accuracy: 

4142/4142 [==============================] - 12s 3ms/step - loss: 0.6729 - accuracy: 0.5932 - val_loss: 0.6772 - val_accuracy: 0.5779
Epoch 57/200
4142/4142 [==============================] - 12s 3ms/step - loss: 0.6724 - accuracy: 0.5931 - val_loss: 0.6756 - val_accuracy: 0.5855
Epoch 58/200
4142/4142 [==============================] - 13s 3ms/step - loss: 0.6723 - accuracy: 0.5943 - val_loss: 0.6737 - val_accuracy: 0.5889
Epoch 59/200
4142/4142 [==============================] - 12s 3ms/step - loss: 0.6715 - accuracy: 0.5964 - val_loss: 0.6732 - val_accuracy: 0.5895
Epoch 60/200
4142/4142 [==============================] - 12s 3ms/step - loss: 0.6702 - accuracy: 0.5992 - val_loss: 0.6729 - val_accuracy: 0.5863
Epoch 61/200
4142/4142 [==============================] - 12s 3ms/step - loss: 0.6698 - accuracy: 0.6004 - val_loss: 0.6727 - val_accuracy: 0.5926
Epoch 62/200
4142/4142 [==============================] - 12s 3ms/step - loss: 0.6685 - accuracy: 0.5990 - val_loss: 0.6710 - val_a

Epoch 112/200
4142/4142 [==============================] - 15s 4ms/step - loss: 0.6260 - accuracy: 0.6575 - val_loss: 0.6283 - val_accuracy: 0.6553
Epoch 113/200
4142/4142 [==============================] - 15s 4ms/step - loss: 0.6265 - accuracy: 0.6567 - val_loss: 0.6296 - val_accuracy: 0.6507
Epoch 114/200
4142/4142 [==============================] - 14s 3ms/step - loss: 0.6253 - accuracy: 0.6593 - val_loss: 0.6284 - val_accuracy: 0.6588
Epoch 115/200
4142/4142 [==============================] - 15s 4ms/step - loss: 0.6242 - accuracy: 0.6598 - val_loss: 0.6299 - val_accuracy: 0.6526
Epoch 116/200
4142/4142 [==============================] - 15s 4ms/step - loss: 0.6253 - accuracy: 0.6590 - val_loss: 0.6269 - val_accuracy: 0.6578
Epoch 117/200
4142/4142 [==============================] - 15s 4ms/step - loss: 0.6247 - accuracy: 0.6580 - val_loss: 0.6218 - val_accuracy: 0.6618
Epoch 118/200
4142/4142 [==============================] - 15s 4ms/step - loss: 0.6251 - accuracy: 0.6604 - val_

4142/4142 [==============================] - 17s 4ms/step - loss: 0.6161 - accuracy: 0.6707 - val_loss: 0.6241 - val_accuracy: 0.6557
Epoch 168/200
4142/4142 [==============================] - 18s 4ms/step - loss: 0.6164 - accuracy: 0.6686 - val_loss: 0.6354 - val_accuracy: 0.6480
Epoch 169/200
4142/4142 [==============================] - 20s 5ms/step - loss: 0.6165 - accuracy: 0.6716 - val_loss: 0.6199 - val_accuracy: 0.6673
Epoch 170/200
4142/4142 [==============================] - 18s 4ms/step - loss: 0.6162 - accuracy: 0.6691 - val_loss: 0.6196 - val_accuracy: 0.6622
Epoch 171/200
4142/4142 [==============================] - 17s 4ms/step - loss: 0.6169 - accuracy: 0.6680 - val_loss: 0.6219 - val_accuracy: 0.6644
Epoch 172/200
4142/4142 [==============================] - 19s 5ms/step - loss: 0.6157 - accuracy: 0.6706 - val_loss: 0.6186 - val_accuracy: 0.6679
Epoch 173/200
4142/4142 [==============================] - 19s 5ms/step - loss: 0.6154 - accuracy: 0.6707 - val_loss: 0.6249 -

In [4]:
%load_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 13764), started 4 days, 4:38:01 ago. (Use '!kill 13764' to kill it.)

In [ ]:
def analyze():
    gods = dbConn.select("SELECT * FROM God ORDER BY name", ())
    X, y = get_data(gods)
    model = load_model('./models/dnn.h5')
    model.summary()

    # evaluate the keras model
    _, accuracy = model.evaluate(X, y)
    print('Accuracy: %.2f' % (accuracy * 100))

    y_krm = model.predict(X)
    mse_krm = mean_squared_error(y, y_krm)
    print(mse_krm)
    
analyze()

In [ ]:
def predict():
    model = load_model('./models/dnn.h5')

    gods = dbConn.select("SELECT * FROM God ORDER BY name", ())
    matches = dbConn.select("SELECT "
                            "t1_god1, t1_god2, t1_god3, t1_god4, t1_god5, "
                            "t2_god1, t2_god2, t2_god3, t2_god4, t2_god5, "
                            "ban1, ban2, ban3, ban4, ban5, ban6, ban7, ban8, ban9, ban10, "
                            "win FROM Match", ())

    win_good_with = get_win_good_with(gods, matches)
    win_good_against = get_win_good_against(gods, matches)

    team_1 = ('Arachne', 'Aphrodite', 'Artemis', 'Cerberus', 'Freya')
    team_2 = ('Bakasura', 'Hel', 'Agni', 'Thoth', 'Discordia')
    features = list(map(lambda god: 1 if god in match[:5] else -1 if god in match[5:10] else 0, gods))
    features_team_1 = list(map(lambda god: 1 if god[1] in team_1 else 0, gods))
    features_team_2 = list(map(lambda god: 1 if god[1] in team_2 else 0, gods))
    features = features_team_1 + features_team_2

    gods = list(map(lambda x: x[1], gods))

    s_team_1 = 0
    s_team_2 = 0
    n_synergy_team_1 = 0
    n_synergy_team_2 = 0
    counter = 0
    n_counter = 0
    for god_1_name in team_1:
        i = gods.index(god_1_name)
        for god_2_name in team_1:
            if god_1_name != god_2_name:
                j = gods.index(god_2_name)
                s_team_1 += win_good_with[i, j] if win_good_with[i, j] > 0 else 0.5
                n_synergy_team_1 += 1

    for god_1_name in team_2:
        i = gods.index(god_1_name)
        for god_2_name in team_2:
            if god_1_name != god_2_name:
                j = gods.index(god_2_name)
                s_team_2 += win_good_with[i, j] if win_good_with[i, j] > 0 else 0.5
                n_synergy_team_2 += 1

        for god_2_name in team_1:
            if god_1_name != god_2_name:
                j = gods.index(god_2_name)
                counter += win_good_against[i, j]
                n_counter += 1

    s_team_1 = s_team_1 / n_synergy_team_1
    s_team_2 = s_team_2 / n_synergy_team_2
    synergy = s_team_1 / (s_team_1 + s_team_2)
    features.append(synergy)
    features.append(counter / n_counter)

    print("synergy team 1", s_team_1)
    print("synergy team 2", s_team_2)
    print("synergy", synergy)
    print("counter", counter / n_counter)
    print(features)

    # make probability predictions with the model
    predictions = model.predict([features])
    print(predictions)

    # make class predictions with the model
    predictions = (predictions > 0.5).astype(int)
    print(predictions)
    
predict()